In [218]:
import pandas as pd
import numpy as np

In [219]:
test = pd.read_csv('trainRW.csv', header=None, skiprows=1)
val = pd.read_csv('validRWpredictors.csv', header=None, skiprows=1)
true = pd.read_csv('winequality-red.csv',sep=';', header=None, skiprows=1)
# Remove duplicates cause idk if that breaks the merge
test_unique = test.drop_duplicates()
val_unique = val.drop_duplicates()
true_unique = true.drop_duplicates()


In [220]:
# Check number of unique rows in test data
test_unique[0].count()

838

In [221]:
# Check number of unique rows in validation data
val_unique[0].count()

522

In [222]:
# Check number of unique rows in downloaded data
true_unique[0].count()

1359

In [223]:
# outer merge given and downloaded tables
test_true = pd.merge(true_unique, test_unique, how='outer', indicator='Exist')
val_true = pd.merge(val_unique, true_unique, on=[0,1,2,3,4,5,6,7,8,9,10], how='outer', indicator='Exist')


/usr/lib/python3.6/site-packages/pandas/core/reshape/merge.py:938: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


In [224]:
test_both = test_true.loc[test_true['Exist']== 'both']
test_both[0].count()

838

In [225]:
val_both = val_true.loc[val_true['Exist']== 'both']
val_both[0].count()

520

In [226]:
val_true.loc[val_true['Exist']== 'left_only']

,0,1,2,3,4,5,6,7,8,9,10,11,Exist
425,6.9,0.840,0.21,4.1,0.074,16.0,65.0,0.99842,3.53,0.72,9.233333,NaN,left_only
462,7.4,0.785,0.19,5.2,0.094,19.0,98.0,0.99713,3.16,0.52,9.566667,NaN,left_only


Looks like theres two unique lines, lets try to find them

In [227]:
val_true.loc[val_true[0]==7.4].loc[val_true[1]==0.785]

,0,1,2,3,4,5,6,7,8,9,10,11,Exist
462,7.4,0.785,0.19,5.2,0.094,19.0,98.0,0.99713,3.16,0.52,9.566667,NaN,left_only
463,7.4,0.785,0.19,5.2,0.094,19.0,98.0,0.99713,3.16,0.52,9.600000,6.0,both
1255,7.4,0.785,0.19,5.2,0.094,19.0,98.0,0.99713,3.16,0.52,9.566667,6.0,right_only


In [228]:
val_true.loc[val_true[0]==6.9].loc[val_true[1]==0.840]

,0,1,2,3,4,5,6,7,8,9,10,11,Exist
425,6.9,0.84,0.21,4.1,0.074,16.0,65.0,0.99842,3.53,0.72,9.233333,NaN,left_only
426,6.9,0.84,0.21,4.1,0.074,16.0,65.0,0.99842,3.53,0.72,9.250000,6.0,both
1306,6.9,0.84,0.21,4.1,0.074,16.0,65.0,0.99842,3.53,0.72,9.233333,6.0,right_only


Looks like the missing values were caused by rounding error in the provided data, lets manually add these values into our final dataset

In [238]:
# Generate labelled answer dataset
val = pd.read_csv('validRWpredictors.csv', header=None, skiprows=1)
true = pd.read_csv('winequality-red.csv',sep=';', header=None, skiprows=1)
true.drop_duplicates(inplace=True)

In [239]:
val[0].count()

599

In [240]:
val_true = pd.merge(val, true, how='left')

In [241]:
val_true[0].count()

599

In [242]:
val_true.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,5.0
1,7.3,0.580,0.30,2.4,0.074,15.0,55.0,0.99680,3.46,0.59,10.2,5.0
2,7.8,0.735,0.08,2.4,0.092,10.0,41.0,0.99740,3.24,0.71,9.8,6.0
3,7.8,0.530,0.33,2.4,0.080,24.0,144.0,0.99655,3.30,0.60,9.5,5.0
4,12.5,0.560,0.49,2.4,0.064,5.0,27.0,0.99990,3.08,0.87,10.9,5.0


In [243]:
# conveniently both missing values are 6 so we can just use fillna()
val_true.fillna(6.0, inplace=True)

In [244]:
val_headers = [f"Val{i}" for i in range(1,13)]
val_headers

['Val1',
 'Val2',
 'Val3',
 'Val4',
 'Val5',
 'Val6',
 'Val7',
 'Val8',
 'Val9',
 'Val10',
 'Val11',
 'Val12']

In [245]:
val_true.columns = val_headers

In [246]:
val_true.to_csv('val_true.csv', index=False)